In [1]:
import pandas as pd
import seaborn as sns
from pathlib import Path
from scipy.stats import t
import numpy as np

In [70]:
# Load the data
output_path = Path('../outputs/20240712-115425-manhattan_lp_fixed2')

data = pd.read_csv(output_path / 'results.csv')
data.fillna(0, inplace=True)

data['algorithm'] = data.agent + '_t' + data.time_limit.astype(str)
data.loc[data.agent == 'LP', 'algorithm'] = 'LP'
data['benchmark'] = data.env + '_' + data.instance + '_cons_thd=' + \
                    data.cons_thd.astype(str) + '_p=' + data.period.astype(str) + '_rad=' + data.radius.astype(str)

data['feasible'] = data.penalty_mean <= data.c

data['t'] = (data['penalty_mean'] - data['c']) * np.sqrt(data['repetitions']) / data['penalty_std']
data.loc[data['penalty_std'] == 0, 't'] = 0

data['feasible_low'] = data['t'] <= t(df=data['repetitions']-1).ppf(0.95)
data['feasible_high'] = data['t'] <= t(df=data['repetitions']-1).ppf(0.05)
data.loc[data.agent == 'LP', 'feasible_low'] = True
data.loc[data.agent == 'LP', 'feasible_high'] = True

data.head()
data['feasible_low'].value_counts()
data[data.c == 10]['feasible'].value_counts()

0       Morningside
1       Morningside
2       Morningside
3       Morningside
4       Morningside
           ...     
1615         Harlem
1616         Harlem
1617         Harlem
1618         Harlem
1619         Harlem
Name: instance, Length: 1620, dtype: object

### Tables

In [78]:
import warnings
warnings.filterwarnings("ignore")

def get_table(data):
    # get sum of rewards on benchmarks we admit as solved
    feasible_data = data[data['feasible_low'] == True]
    reward_sum_by_algorithm = feasible_data.groupby('algorithm')['reward_mean'].sum().reset_index()

    # get total number of runs of each algorithm (is the same value for all algorithms)
    total_rows_by_algorithm = data['algorithm'].value_counts().reset_index()
    total_rows_by_algorithm.columns = ['algorithm', 'total_rows']

    reward_sum_by_algorithm = pd.merge(reward_sum_by_algorithm, total_rows_by_algorithm, on='algorithm')
    reward_sum_by_algorithm['cr'] = reward_sum_by_algorithm['reward_mean'] / reward_sum_by_algorithm['total_rows']
    reward_result = reward_sum_by_algorithm[['algorithm', 'cr']]

    aggregate_table = data.groupby('algorithm').agg(
        esat=pd.NamedAgg(column='feasible', aggfunc='sum'),
        sat=pd.NamedAgg(column='feasible_low', aggfunc='sum'),
        strongsat=pd.NamedAgg(column='feasible_high', aggfunc='sum'),
    ).reset_index()

    final_result = pd.merge(reward_result, aggregate_table, on='algorithm', how='outer')

    return final_result.sort_values(by='algorithm')

tables = {
    'table.csv': get_table(data),
}


for instance in data.instance.unique():
    print(instance + "\n", get_table(data[data.instance == instance]))
for name, table in tables.items():
    table.to_csv(output_path / name)
    print('\n', name, '\n', table)


warnings.filterwarnings("default")


Morningside
          algorithm        cr  esat  sat  strongsat
0    DualUCT_t1000  0.018609    24   24          6
1     DualUCT_t200  0.015832    24   24          6
2     DualUCT_t500  0.027219    24   24          6
3  ParetoUCT_t1000  6.784872    21   21         15
4   ParetoUCT_t200  7.184556    25   28         23
5   ParetoUCT_t500  6.958189    21   23         18
6      RAMCP_t1000  6.434356    31   31         30
7       RAMCP_t200  5.531113    33   33         32
8       RAMCP_t500  5.881078    31   32         31
Lexington
          algorithm        cr  esat  sat  strongsat
0    DualUCT_t1000  1.013782    12   12         12
1     DualUCT_t200  0.877128    10   11          8
2     DualUCT_t500  1.004338    12   12         10
3  ParetoUCT_t1000  5.348625    22   24         19
4   ParetoUCT_t200  5.158923    27   27         24
5   ParetoUCT_t500  5.641095    23   26         21
6      RAMCP_t1000  4.641480    31   31         30
7       RAMCP_t200  4.787576    32   34         32
8      

### Plots

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

# Plot meand reward vs. c
# Each benchmark is a subplot
# Each algorithm is a line
# Use `reward_std` for error bars


for time in [t for t in data['time_limit'].unique() if t >= 0]:
    filtered_data = data[(data['time_limit'] == time) | (data['time_limit'] == -1)]
    filtered_data['reward_std'] /= np.sqrt(filtered_data['repetitions'])

    default_palette = sns.color_palette()
    algorithms = filtered_data['algorithm'].unique()
    palette = {algorithm: default_palette[i] for i, algorithm in enumerate(algorithms)}


    # Create a FacetGrid with the benchmark feature
    g = sns.FacetGrid(filtered_data, col="benchmark", col_wrap=4, height=4, aspect=1.5)

    # Do not show the plot in a notebook
    plt.ioff()

    # Iterate through each subplot to add error bars manually
    for ax, (benchmark_value, subset) in zip(g.axes.flatten(), filtered_data.sort_values(by='instance').groupby('benchmark')):
        sns.lineplot(x='c', y='reward_mean', hue='algorithm', palette=palette, err_style=None, data=subset, ax=ax)

        non_feasible = subset[subset['feasible_low'] == False]
        sns.scatterplot(x='c', y='reward_mean', hue='algorithm', palette=palette, marker='x', s=50, data=non_feasible, ax=ax)
        ax.grid(True)

        for alg in algorithms:
            alg_data = subset[subset['algorithm'] == alg]
            ax.fill_between(
                alg_data['c'],
                alg_data['reward_mean'] - alg_data['reward_std'],
                alg_data['reward_mean'] + alg_data['reward_std'],
                alpha=0.2,
                color=palette[alg]
            )

    # Adjust the titles and labels
    g.set_titles(col_template="{col_name}")
    g.set_axis_labels("C", "Mean Reward")

    g.savefig(output_path / f'mean_reward_vs_c_t{time}.svg')


warnings.filterwarnings("default")